### DCGAN

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from tqdm import tqdm

In [ ]:
def weights_init(w):
    """
    Initializes the weights of the layer, w.
    """
    classname = w.__class__.__name__
    if classname.find('conv') != -1:
        nn.init.normal_(w.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(w.weight.data, 1.0, 0.02)
        nn.init.constant_(w.bias.data, 0)
        

In [ ]:
class Generator(nn.Module):
    def __init__(self, params):
        super(Generator, self).__init__()
        nz, ngf, nc = params['nz'], params['ngf'], params['nc']

        self.main = nn.Sequential(
            # Transposed Convolution Block 1
            nn.ConvTranspose2d(nz, ngf * 16, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 16),
            nn.ReLU(True),
            
            # Transposed Convolution Block 2
            nn.ConvTranspose2d(ngf * 16, ngf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),

            # Transposed Convolution Block 2
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),

            # Transposed Convolution Block 3
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),

            # Transposed Convolution Block 4
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),

            # Final Transposed Convolution Block
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)


In [ ]:
class Discriminator(nn.Module):
    def __init__(self, params):
        super(Discriminator, self).__init__()
        nc, ndf = params['nc'], params['ndf']

        self.main = nn.Sequential(
            # Convolution Block 1
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            # Convolution Block 2
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),

            # Convolution Block 3
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),

            # Convolution Block 4
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Convolution Block 4
            nn.Conv2d(ndf * 8, ndf * 16, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 16),
            nn.LeakyReLU(0.2, inplace=True),

            # Final Convolution Layer
            nn.Conv2d(ndf * 16, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x)


In [ ]:
# Parameters to define the model.
params = {
    "bsize" : 128,# Batch size during training.
    'imsize' : 128,# Spatial size of training images. All images will be resized to this size during preprocessing.
    'nc' : 3,# Number of channles in the training images. For coloured images this is 3.
    'nz' : 100,# Size of the Z latent vector (the input to the generator).
    'ngf' : 64,# Size of feature maps in the generator. The depth will be multiples of this.
    'ndf' : 64, # Size of features maps in the discriminator. The depth will be multiples of this.
    'nepochs' : 200,# Number of training epochs.
    'lr' : 0.0002,# Learning rate for optimizers
    'beta1' : 0.5,# Beta1 hyperparam for Adam optimizer
    'save_epoch' : 2}# Save step.


In [ ]:
device = torch.device("cuda:1" if(torch.cuda.is_available()) else "cpu")
print(device, " will be used.\n")

In [ ]:
# Define the path to your dataset and other hyperparameters
data_dir_train = '/data4/home/anujkumar1/GAN/Datasets/images/train' 
data_dir_val = '/data4/home/anujkumar1/GAN/Datasets/images/val'
shuffle_dataset = True  
num_workers = 4  


transform = transforms.Compose([
    transforms.Resize((params['imsize'],params['imsize'])), 
    transforms.ToTensor(),  
])


train_dataset = ImageFolder(root= data_dir_train, transform=transform)
val_dataset = ImageFolder(root= data_dir_val , transform=transform)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=params['bsize'],
    shuffle=shuffle_dataset,
    num_workers=num_workers
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=100,
    shuffle=shuffle_dataset,
    num_workers=num_workers
)


In [ ]:
# Plot the training images.
sample_batch = next(iter(val_dataloader))
plt.figure(figsize=(8, 8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(
    sample_batch[0].to(device)[ : 64], padding=2, normalize=True).cpu(), (1, 2, 0)))

plt.show()

In [ ]:
# Create the generator.
netG = Generator(params).to(device)
# Apply the weights_init() function to randomly initialize all
# weights to mean=0.0, stddev=0.2
netG.apply(weights_init)
# Print the model.
print(netG)

In [ ]:
# Create the discriminator.
netD = Discriminator(params).to(device)
# Apply the weights_init() function to randomly initialize all
# weights to mean=0.0, stddev=0.2
netD.apply(weights_init)
# Print the model.
print(netD)

In [ ]:
# Binary Cross Entropy loss function.
criterion = nn.BCELoss()

In [ ]:
fixed_noise = torch.randn(64, params['nz'], 1, 1, device=device)

real_label = 1.0
fake_label = 0.0


In [ ]:
# Optimizer for the discriminator.
optimizerD = optim.Adam(netD.parameters(), lr=params['lr'], betas=(params['beta1'], 0.999))
# Optimizer for the generator.
optimizerG = optim.Adam(netG.parameters(), lr=params['lr'], betas=(params['beta1'], 0.999))

In [ ]:
img_list = []
# Stores generator losses during training.
G_losses = []
# Stores discriminator losses during training.
D_losses = []

iters = 0


In [ ]:
# Initialize lists to track generator and discriminator losses
G_losses = []
D_losses = []

# Initialize a list to store generated images for visualization
img_list = []

# Training loop
for epoch in range(params['nepochs']):
    data_iterator = tqdm(enumerate(train_dataloader, 0), total=len(train_dataloader), desc=f"Epoch {epoch+1}")
    for i, data in data_iterator:
        # Transfer data tensor to GPU/CPU (device)
        real_data = data[0].to(device)
        b_size = real_data.size(0)

        # Discriminator training
        netD.zero_grad()
        label = torch.full((b_size, ), real_label, device=device)
        output = netD(real_data).view(-1)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        noise = torch.randn(b_size, params['nz'], 1, 1, device=device)
        fake_data = netG(noise)
        label.fill_(fake_label)
        output = netD(fake_data.detach()).view(-1)
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()

        errD = errD_real + errD_fake
        optimizerD.step()

        # Generator training
        netG.zero_grad()
        label.fill_(real_label)
        output = netD(fake_data).view(-1)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Visualize generated images
        if (iters % 100 == 0) or ((epoch == params['nepochs']-1) and (i == len(train_dataloader)-1)):
            with torch.no_grad():
                fake_data = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake_data, padding=2, normalize=True))

        iters += 1
        
        data_iterator.set_description(f"Epoch {epoch+1}, Loss_D: {errD.item():.4f}, Loss_G: {errG.item():.4f}")
    # Check progress
        # if i % 50 == 0:
    print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
            % (epoch+1, params['nepochs'], i, len(train_dataloader),
                errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
    
    if (epoch+1)%50 == 0:
        noise = torch.randn(100, params['nz'], 1, 1, device=device)
        # Generate images
        with torch.no_grad():
            generated_imgs = netG(noise).detach().cpu()

        # Reshape the generated images into a 10x10 grid
        grid = vutils.make_grid(generated_imgs, nrow= 10, padding=2, normalize=True)

        # Display the generated image grid with a larger size
        plt.figure(figsize=(12, 12))  # Adjust the figsize as needed
        plt.axis("off")
        plt.title("Generated Images")
        plt.imshow(np.transpose(grid, (1, 2, 0)))
        plt.show()


In [ ]:
# Plot the training losses.
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()



In [ ]:
# # Save the Generator model
# torch.save(netG.state_dict(), 'DCGAN_generator_model.pth')

# # Save the Discriminator model
# torch.save(netD.state_dict(), 'DCGAN_discriminator_model.pth')


In [ ]:
noise = torch.randn(100, params['nz'], 1, 1, device=device)
# Generate images
with torch.no_grad():
    generated_imgs = netG(noise).detach().cpu()

# Reshape the generated images into a 10x10 grid
grid = vutils.make_grid(generated_imgs, nrow= 10, padding=2, normalize=True)

# Display the generated image grid with a larger size
plt.figure(figsize=(12, 12))  # Adjust the figsize as needed
plt.axis("off")
plt.title("Generated Images")
plt.imshow(np.transpose(grid, (1, 2, 0)))
plt.show()

In [ ]:
noise = torch.randn(100, params['nz'], 1, 1, device=device)
# Generate images
with torch.no_grad():
    generated_imgs = netG(noise).detach().cpu()

# Reshape the generated images into a 10x10 grid
grid = vutils.make_grid(generated_imgs, nrow=10, padding=2, normalize=True)

# Plot the training images.
sample_batch = next(iter(val_dataloader))

# Create subplots for generated images and training images side by side
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Display the training images
axes[0].axis("off")
axes[0].set_title("Real Images")
axes[0].imshow(np.transpose(vutils.make_grid(
    sample_batch[0].to(device)[:100],nrow=10, padding=2, normalize=True).cpu(), (1, 2, 0)))

# Display the generated image grid
axes[1].axis("off")
axes[1].set_title("Generated Images")
axes[1].imshow(np.transpose(grid, (1, 2, 0)))


plt.show()


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np
from scipy.linalg import sqrtm

# Load a pre-trained Inception model
from torchvision.models import inception_v3

# Load the Inception-V3 model without using deprecated parameters
inception_model = inception_v3(pretrained=True, num_classes=1000)
inception_model.aux_logits = False  # Disable auxiliary logits
inception_model.fc = nn.Identity()  # Remove the final classification layer

def extract_inception_features(images, batch_size=32):
    transform = transforms.Compose([transforms.Resize((299, 299))])
    dataset = torch.utils.data.TensorDataset(images, torch.zeros(len(images)))  # Create a dummy target
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    inception_features = []

    inception_model.eval()
    with torch.no_grad():
        for batch in dataloader:
            batch_images, _ = batch
            batch_images = transform(batch_images)  # Resize the images to 299x299
            batch_features = inception_model(batch_images)
            inception_features.append(batch_features)

    inception_features = torch.cat(inception_features, 0)
    return inception_features.numpy()


# Function to calculate FID
def calculate_fid(real_features, gen_features):
    mu_real = np.mean(real_features, axis=0)
    mu_gen = np.mean(gen_features, axis=0)
    
    cov_real = np.cov(real_features, rowvar=False)
    cov_gen = np.cov(gen_features, rowvar=False)
    
    cov_sqrt = sqrtm(np.dot(cov_real, cov_gen))
    
    fid = np.sum((mu_real - mu_gen) ** 2) + np.trace(cov_real + cov_gen - 2 * cov_sqrt)
    
    return fid



In [ ]:
noise = torch.randn(1000, params['nz'], 1, 1, device=device)
# Generate images
with torch.no_grad():
    generated_imgs = netG(noise).detach().cpu()
    
real = []
real_labels = []

for batch_images, batch_labels in val_dataloader:
    # Append the batch of images and labels to the collected lists
    real.append(batch_images)
    real_labels.append(batch_labels)

    # Check if you have collected enough images
    num_collected_images = sum(len(images) for images in real)
    if num_collected_images >= 1000:
        break


real_imgs = torch.cat(real)[:1000]
real_labels = torch.cat(real_labels)[:1000]
real_imgs = real_imgs.detach().cpu()


In [ ]:
# Assuming real_images and gen_images are PyTorch tensors or numpy arrays
# Extract features from real and generated images
real_features = extract_inception_features(real_imgs)
gen_features = extract_inception_features(generated_imgs)

# Calculate FID
fid = calculate_fid(real_features, gen_features)


In [ ]:
print(f"FID: {np.real(fid)}")

1. **Training Setup:**
   - DCGAN trained on provided data with standard GAN loss.
   - Varying training iterations for both generator and discriminator.

2. **Loss Analysis:**
   - Studied loss curves for generator and discriminator.
   - Observed convergence patterns and behaviors during training.

3. **Generated Images:**
   - Presented a 10x10 grid of generated images.
   - Noted any distinctive features or patterns in the images.

4. **FID Score:**
   - Calculated FID score of 48.00345601901948.
   - FID used to assess likeness between real and generated images.

5. **Observations:**
   - Varied training iterations impacted model performance.
   - Reflect on the implications of FID score in evaluating image quality.


#### Multiple number of Discriminator Iteration

In [ ]:
num_iter = [2,3,5]   

for iter in num_iter:
    # Create the generator.
    netG = Generator(params).to(device)
    # Apply the weights_init() function to randomly initialize all
    # weights to mean=0.0, stddev=0.2
    netG.apply(weights_init)
    # Create the discriminator.
    netD = Discriminator(params).to(device)
    # Apply the weights_init() function to randomly initialize all
    # weights to mean=0.0, stddev=0.2
    netD.apply(weights_init)
    # Binary Cross Entropy loss function.
    criterion = nn.BCELoss()
    fixed_noise = torch.randn(64, params['nz'], 1, 1, device=device)

    real_label = 1.0
    fake_label = 0.0

    # Optimizer for the discriminator.
    optimizerD = optim.Adam(netD.parameters(), lr=params['lr'], betas=(params['beta1'], 0.999))
    # Optimizer for the generator.
    optimizerG = optim.Adam(netG.parameters(), lr=params['lr'], betas=(params['beta1'], 0.999))

    # Initialize lists to track generator and discriminator losses
    G_losses = []
    D_losses = []
    iters = 0
    # Initialize a list to store generated images for visualization
    img_list = []

    # Training loop
    for epoch in range(params['nepochs']):
        # data_iterator = tqdm(enumerate(train_dataloader, 0), total=len(train_dataloader), desc=f"Epoch {epoch+1}")
        for  data,_ in train_dataloader:
            # Transfer data tensor to GPU/CPU (device)
            real_data = data.to(device)
            b_size = real_data.size(0)

            for _ in range(iter):    
                # Discriminator training
                netD.zero_grad()
                label = torch.full((b_size, ), real_label, device=device)
                output = netD(real_data).view(-1)
                errD_real = criterion(output, label)
                errD_real.backward()
                D_x = output.mean().item()

                noise = torch.randn(b_size, params['nz'], 1, 1, device=device)
                fake_data = netG(noise)
                label.fill_(fake_label)
                output = netD(fake_data.detach()).view(-1)
                errD_fake = criterion(output, label)
                errD_fake.backward()
                D_G_z1 = output.mean().item()

                errD = errD_real + errD_fake
                optimizerD.step()

            # Generator training
            netG.zero_grad()
            label.fill_(real_label)
            output = netD(fake_data).view(-1)
            errG = criterion(output, label)
            errG.backward()
            D_G_z2 = output.mean().item()
            optimizerG.step()

            
            G_losses.append(errG.item())
            D_losses.append(errD.item())

            # # Visualize generated images
            # if (iters % 100 == 0) or ((epoch == params['nepochs']-1) and (i == len(train_dataloader)-1)):
            #     with torch.no_grad():
            #         fake_data = netG(fixed_noise).detach().cpu()
            #     img_list.append(vutils.make_grid(fake_data, padding=2, normalize=True))

            iters += 1
            
            # data_iterator.set_description(f"Epoch {epoch+1}, Loss_D: {errD.item():.4f}, Loss_G: {errG.item():.4f}")
        # # Check progress
        # if (epoch+1) % 10 == 0:
        #     print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
        #             % (epoch+1, params['nepochs'], len(train_dataloader),
        #                 errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
            
        if (epoch+1)%50 == 0:
            noise = torch.randn(100, params['nz'], 1, 1, device=device)
            # Generate images
            with torch.no_grad():
                generated_imgs = netG(noise).detach().cpu()

            # Reshape the generated images into a 10x10 grid
            grid = vutils.make_grid(generated_imgs, nrow= 10, padding=2, normalize=True)

            # Display the generated image grid with a larger size
            plt.figure(figsize=(12, 12))  # Adjust the figsize as needed
            plt.axis("off")
            plt.title(f"Generated Images (iter {iter})")
            plt.imshow(np.transpose(grid, (1, 2, 0)))
            plt.show()


    # Plot the training losses.
    plt.figure(figsize=(10,5))
    plt.title(f"Generator and Discriminator Loss During Training (iter {iter})")
    plt.plot(G_losses,label="G")
    plt.plot(D_losses,label="D")
    plt.xlabel("iterations")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

    # Save the Generator model
    torch.save(netG.state_dict(), f'DCGAN_generator_model_{iter}.pth')

    # Save the Discriminator model
    torch.save(netD.state_dict(), f'DCGAN_discriminator_model_{iter}.pth')


    

In [ ]:
for iter in num_iter:
    # Load the Generator model
    dis_path  = f'/data4/home/anujkumar1/GAN/DCGAN_discriminator_model_{iter}.pth'
    gen_path = f'/data4/home/anujkumar1/GAN/DCGAN_generator_model_{iter}.pth'
    netG.load_state_dict(torch.load(gen_path))
    netG.eval()

    # Load the Discriminator model
    netD.load_state_dict(torch.load(dis_path))
    netD.eval()

        

    noise = torch.randn(100, params['nz'], 1, 1, device=device)
    # Generate images
    with torch.no_grad():
        generated_imgs = netG(noise).detach().cpu()

    # Reshape the generated images into a 10x10 grid
    grid = vutils.make_grid(generated_imgs, nrow= 10, padding=2, normalize=True)

    # Display the generated image grid with a larger size
    plt.figure(figsize=(12, 12))  # Adjust the figsize as needed
    plt.axis("off")
    plt.title(f"Generated Images (iter {iter})")
    plt.imshow(np.transpose(grid, (1, 2, 0)))
    plt.show()

    noise = torch.randn(1000, params['nz'], 1, 1, device=device)
    # Generate images
    with torch.no_grad():
        generated_imgs = netG(noise).detach().cpu()
        
    real = []
    real_labels = []

    for batch_images, batch_labels in val_dataloader:
        # Append the batch of images and labels to the collected lists
        real.append(batch_images)
        real_labels.append(batch_labels)

        # Check if you have collected enough images
        num_collected_images = sum(len(images) for images in real)
        if num_collected_images >= 1000:
            break


    real_imgs = torch.cat(real)[:1000]
    real_labels = torch.cat(real_labels)[:1000]
    real_imgs = real_imgs.detach().cpu()

    # Assuming real_images and gen_images are PyTorch tensors or numpy arrays
    # Extract features from real and generated images
    real_features = extract_inception_features(real_imgs)
    gen_features = extract_inception_features(generated_imgs)

    # Calculate FID
    fid = calculate_fid(real_features, gen_features)

    print(f"FID (iter {iter}): {np.real(fid)}")

1. **Training Iterations Impact:**
   - Generator and discriminator trained 2, 3, and 5 times independently.
   - Observed behavior changes in loss convergence and image generation.

2. **FID Scores Analysis:**
   - FID (iter 2): 68.0875, FID (iter 3): 92.4330, FID (iter 5): 68.4888.
   - Notable fluctuations in FID scores with varying training iterations.

3. **Behavioral Changes:**
   - Increased training iterations led to improved stability (or degradation) in convergence.
   - Analyzed the impact on the visual quality of generated images.
   - There is repetion of image is observed when we increased the training of discriminator

4. **Consideration for Optimal Iterations:**
   - Evaluate trade-offs between training efficiency and image fidelity.
   - Discuss implications for selecting the optimal number of iterations.


### CWGAN with LABEL as INPUT

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.transforms as transformations
from torch.utils.data import DataLoader
import torchvision.utils as vutils
import PIL.Image as Image
import torchvision.models as models
from torchvision.datasets import ImageFolder
import numpy as np
from tqdm import tqdm
from ignite.metrics.gan import FID
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Parameters to define the model.
params = {
    "bsize" : 64,# Batch size during training.
    'imsize' : 128,# Spatial size of training images. All images will be resized to this size during preprocessing.
    'nc' : 3,# Number of channles in the training images. For coloured images this is 3.
    'nz' : 100,# Size of the Z latent vector (the input to the generator).
    'ngf' : 64,# Size of feature maps in the generator. The depth will be multiples of this.
    'ndf' : 64, # Size of features maps in the discriminator. The depth will be multiples of this.
    'nepochs' : 200,# Number of training epochs.
    'lr' : 1e-4,# Learning rate for optimizers
    'beta1' : 0.5,# Beta1 hyperparam for Adam optimizer
    'num_classes' : 3,# number of label in animal dataset
    'gen_embs' : 100, # embedding of labels
    'critic_iter' : 5, # number of critic iteration
    'lambda_gp' : 10 # Gradient penality
    } 


In [ ]:
device = torch.device("cuda:2" if(torch.cuda.is_available()) else "cpu")
print(device, " will be used.\n")

In [ ]:
# Define the path to your dataset and other hyperparameters
data_dir_train = '/data4/home/anujkumar1/GAN/Datasets/images/train' 
data_dir_val = '/data4/home/anujkumar1/GAN/Datasets/images/val'
shuffle_dataset = True  
num_workers = 4  


transform = transforms.Compose([
    transforms.Resize((params['imsize'],params['imsize'])), 
    transforms.ToTensor(),  
])


train_dataset = ImageFolder(root= data_dir_train, transform=transform)
val_dataset = ImageFolder(root= data_dir_val , transform=transform)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=params['bsize'],
    shuffle=shuffle_dataset,
    num_workers=num_workers
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=100,
    shuffle=shuffle_dataset,
    num_workers=num_workers
)


In [ ]:
class Generator(nn.Module):
    def __init__(self, params):
        super(Generator, self).__init__()
        self.img_size = params['imsize']
        self.num_classes = params['num_classes']
        self.embed_size = params['gen_embs']
        nz = params['nz']
        ngf = params['ngf']
        nc = params['nc']
        
        self.main = nn.Sequential(
            # Input: N x channels_noise x 1 x 1
            nn.ConvTranspose2d(nz + self.embed_size, ngf * 32, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 32),
            nn.ReLU(),
             # img: 4x4
            nn.ConvTranspose2d(ngf * 32, ngf * 16, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 16),
            nn.ReLU(),
            # img: 8x8
            nn.ConvTranspose2d(ngf * 16, ngf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(),
            # img: 16x16
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(),
            # img: 32x32
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(),
            # img: 64x64
            nn.ConvTranspose2d(ngf * 2, nc, kernel_size=4, stride=2, padding=1),
            # Output: N x channels_img x 128 x 128
            nn.Tanh(),
        )

        self.embed = nn.Embedding(self.num_classes, self.embed_size)

    def forward(self, x, labels):
        embedding = self.embed(labels).unsqueeze(2).unsqueeze(3)
        x = torch.cat([x, embedding], 1)
        return self.main(x)





In [ ]:
class Discriminator(nn.Module):
    def __init__(self, params):
        super(Discriminator, self).__init__()
        self.img_size = params['imsize']
        nc = params['nc']
        ndf = params['ndf']
        num_classes = params['num_classes']

        self.main = nn.Sequential(
            nn.Conv2d(nc + 1, ndf, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),

            # Block 1
            nn.Conv2d(ndf, ndf * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(ndf * 2, affine=True),
            nn.LeakyReLU(0.2),

            # Block 2
            nn.Conv2d(ndf * 2, ndf * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(ndf * 4, affine=True),
            nn.LeakyReLU(0.2),

            # Block 3
            nn.Conv2d(ndf * 4, ndf * 8, kernel_size=4, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(ndf * 8, affine=True),
            nn.LeakyReLU(0.2),

             # Block 4
            nn.Conv2d(ndf * 8, ndf * 16, kernel_size=4, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(ndf * 16, affine=True),
            nn.LeakyReLU(0.2),

            # Output
            nn.Conv2d(ndf * 16, 1, kernel_size=4, stride=2, padding=0),
        )

        # Embedding for conditioning
        self.embed = nn.Embedding(num_classes, self.img_size * self.img_size)

    def forward(self, x, labels):
        embedding = self.embed(labels).view(labels.shape[0], 1, self.img_size, self.img_size)
        x = torch.cat([x, embedding], dim=1)
        return self.main(x)




In [ ]:
def weights_init(w):
    """
    Initializes the weights of the layer, w.
    """
    classname = w.__class__.__name__
    if classname.find('conv') != -1:
        nn.init.normal_(w.weight.data, 0.0, 0.02)
    elif classname.find('bn') != -1:
        nn.init.normal_(w.weight.data, 1.0, 0.02)
        nn.init.constant_(w.bias.data, 0)
        

In [ ]:
# Create the generator model with the provided parameters.
gen = Generator(params).to(device)
gen.apply(weights_init)
critic = Discriminator(params).to(device)
critic.apply(weights_init), gen

In [ ]:
# initializate optimizer
opt_gen = optim.Adam(gen.parameters(), lr=params['lr'], betas=(params['beta1'], 0.999))
opt_critic = optim.Adam(critic.parameters(), lr=params['lr'], betas=(params['beta1'], 0.999))

In [ ]:
fixed_noise = torch.randn(100, params['nz'], 1, 1).to(device)
fixed_labels = torch.randint(0, 3, (100,)).to(device)

In [ ]:
### initialize FID wrapper
fid_score = FID()
#interpolate function to resize images to 299,299,3  which is the input size of inception network
def interpolate(batch):
    arr = []
    for img in batch:
        pil_img = transformations.ToPILImage()(img)
        resized_img = pil_img.resize((299,299), Image.BILINEAR)
        arr.append(transformations.ToTensor()(resized_img))
    return torch.stack(arr)

In [ ]:
# Load the Generator model
gen.load_state_dict(torch.load('/data4/home/anujkumar1/GAN/generator_model.pth'))
# gen.eval()

# Load the Discriminator model
critic.load_state_dict(torch.load('/data4/home/anujkumar1/GAN/critic_model.pth'))
# critic.eval()


In [ ]:
gen.train()
critic.train()
#### gradient penalty function for WGAN-GP
def gradient_penalty(critic,labels, real, fake, device="cpu"):
    BATCH_SIZE, C, H, W = real.shape
    alpha = torch.rand((BATCH_SIZE, 1, 1, 1)).repeat(1, C, H, W).to(device)
    interpolated_images = real * alpha + fake * (1 - alpha)

    # Calculate critic scores
    mixed_scores = critic(interpolated_images, labels)

    # Take the gradient of the scores with respect to the images
    gradient = torch.autograd.grad(
        inputs=interpolated_images,
        outputs=mixed_scores,
        grad_outputs=torch.ones_like(mixed_scores),
        create_graph=True,
        retain_graph=True,
    )[0]
    gradient = gradient.view(gradient.shape[0], -1)
    gradient_norm = gradient.norm(2, dim=1)
    gradient_penalty = torch.mean((gradient_norm - 1) ** 2)
    return gradient_penalty



In [ ]:
step = 0

G_loss = []
C_loss = []

for epoch in range(params['nepochs']):
    #we will track the total loss of the generator and critic for each epoch over the entire dataset
    #initialize the total loss of the generator and critic for each epoch to 0
    total_loss_gen = 0
    total_loss_critic = 0
    
    #move these to device

    #have no gradient for these losse
    # total_loss_gen = torch.no_grad()
    # total_loss_critic = torch.no_grad()
    
    # Target labels 
    for batch_idx, (real, labels) in enumerate(tqdm(train_dataloader)):
        #send labels to device
        labels = labels.to(device)
        batch_step = 0
        real = real.to(device)
        cur_batch_size = real.shape[0]
        
        

        # Train Critic: 
        for _ in range(params['critic_iter']):
            noise = torch.randn(cur_batch_size, params['nz'], 1, 1).to(device)
            if len(noise) != len(labels):
                noise = noise[:len(labels)]
            fake = gen(noise, labels)
            critic_real = critic(real, labels).reshape(-1)
            critic_fake = critic(fake, labels).reshape(-1)
            gp = gradient_penalty(critic, labels, real, fake, device=device)
            loss_critic = (
                -(torch.mean(critic_real) - torch.mean(critic_fake)) + params['lambda_gp'] * gp
            )
            
            
            critic.zero_grad()
            loss_critic.backward(retain_graph=True)
            opt_critic.step()
            
        #trained critic
        

        # Train Generator: 
        gen_fake = critic(fake, labels).reshape(-1)
        loss_gen = -torch.mean(gen_fake)
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()
        
        
        
        
        
        #update the total loss of the generator and critic for each batch in the epoch
        #add just the value no gradients
        #have no gradient for these losse
        #add just the value no gradientsfrom the loss_gen tensor
      
        with torch.no_grad():
            total_loss_gen += loss_gen.item()
            total_loss_critic += loss_critic.item()
        
        
        # # Print losses occasionally and print to tensorboard in a batch
        # if (batch_idx + 1) % 23 == 0 and batch_idx > 0:
        #     print(
        #         f"Epoch [{epoch+1}/{params['nepochs']}] Batch {batch_idx+1}/{len(train_dataloader)} \
        #           Loss D: {loss_critic:.4f}, loss G: {loss_gen:.4f}"
        #     )
            
        batch_step += 1
    
        G_loss.append(loss_gen.item())
        C_loss.append(loss_critic.item())   

    print(f"Epoch [{epoch+1}/{params['nepochs']}] Batch {batch_idx+1}/{len(train_dataloader)} \
                Loss D: {loss_critic:.4f}, loss G: {loss_gen:.4f}"
            )

    if (epoch+1)% 20 == 0:
        
        with torch.no_grad():
            fake = gen(fixed_noise, fixed_labels)
            # take out (up to) 32 examples
            grid = vutils.make_grid(fake, nrow= 10, padding=2, normalize=True)[:100].cpu().numpy()
            # Display the generated image grid with a larger size
            plt.figure(figsize=(12, 12))  # Adjust the figsize as needed
            plt.axis("off")
            plt.title("Generated Images")
            plt.imshow(np.transpose(grid[:100], (1, 2, 0)))
            plt.show()


        #AVERAGE LOSS---

        #get average loss of generator and critic for each epoch
        avg_loss_gen = total_loss_gen / len(train_dataloader)
        avg_loss_critic = total_loss_critic / len(train_dataloader)

    
    step += 1
        

In [ ]:
# Plot the training losses.
plt.figure(figsize=(10,5))
plt.title("Generator and critic Loss During Training")
plt.plot(G_loss,label="G")
plt.plot(C_loss,label="C")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()



In [ ]:
# Save the Generator model
torch.save(gen.state_dict(), 'generator_model.pth')

# Save the Discriminator model
torch.save(critic.state_dict(), 'critic_model.pth')


In [ ]:
# Plot the training losses.
plt.figure(figsize=(10,5))
plt.title("Generator and critic Loss During Training")
plt.plot(G_loss,label="G")
plt.plot(C_loss,label="C")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()



In [ ]:
# # Save the Generator model
# torch.save(gen.state_dict(), 'generator_model.pth')

# # Save the Discriminator model
# torch.save(critic.state_dict(), 'critic_model.pth')


In [ ]:
# Load the Generator model
# gen.load_state_dict(torch.load('/data4/home/anujkumar1/GAN/generator_model.pth'))
gen.eval()

# Load the Discriminator model
# critic.load_state_dict(torch.load('/data4/home/anujkumar1/GAN/critic_model.pth'))
critic.eval()


In [ ]:
fixed_noise = torch.randn(100, params['nz'], 1, 1).to(device)
fixed_labels_0  = torch.full((100,), 0).to(device)
fixed_labels_1  = torch.full((100,), 1).to(device)
fixed_labels_2  = torch.full((100,), 2).to(device)

# Generate images
with torch.no_grad():
    fake_0 = gen(fixed_noise, fixed_labels_0)
    fake_1 = gen(fixed_noise, fixed_labels_1)
    fake_2 = gen(fixed_noise, fixed_labels_2)



In [ ]:
# Reshape the generated images into a 10x10 grid
grid_0 = vutils.make_grid(fake_0, nrow= 10, padding=2, normalize=True)[:100].cpu().numpy()
grid_1 = vutils.make_grid(fake_1, nrow= 10, padding=2, normalize=True)[:100].cpu().numpy()
grid_2 = vutils.make_grid(fake_2, nrow= 10, padding=2, normalize=True)[:100].cpu().numpy()

# Display the generated image grid with a larger size
plt.figure(figsize=(12, 12))  # Adjust the figsize as needed
plt.axis("off")
plt.title("Generated Images_0")
plt.imshow(np.transpose(grid_0[:100], (1, 2, 0)))
plt.show()

# Display the generated image grid with a larger size
plt.figure(figsize=(12, 12))  # Adjust the figsize as needed
plt.axis("off")
plt.title("Generated Images_1")
plt.imshow(np.transpose(grid_1, (1, 2, 0)))
plt.show()

# Display the generated image grid with a larger size
plt.figure(figsize=(12, 12))  # Adjust the figsize as needed
plt.axis("off")
plt.title("Generated Images_2")
plt.imshow(np.transpose(grid_2, (1, 2, 0)))
plt.show()

In [ ]:
fixed_noise = torch.randn(1000, params['nz'], 1, 1).to(device)
fixed_labels_0  = torch.full((1000,), 0).to(device)
fixed_labels_1  = torch.full((1000,), 1).to(device)
fixed_labels_2  = torch.full((1000,), 2).to(device)

# Generate images
with torch.no_grad():
    fake_0 = gen(fixed_noise, fixed_labels_0)
    fake_1 = gen(fixed_noise, fixed_labels_1)
    fake_2 = gen(fixed_noise, fixed_labels_2)



real = []
real_labels = []

for batch_images, batch_labels in val_dataloader:
    # Append the batch of images and labels to the collected lists
    real.append(batch_images)
    real_labels.append(batch_labels)

    # Check if you have collected enough images
    num_collected_images = sum(len(images) for images in real)
    if num_collected_images >= 1000:
        break

# Concatenate the collected images and labels into tensors
real = torch.cat(real)[:1000]
real_images = torch.cat(real_labels)[:1000]

In [ ]:
# Define the two latent vectors for interpolation
latent_vector1 = torch.randn(1, params['nz'], 1, 1).to(device)
latent_vector2 = torch.randn(1, params['nz'], 1, 1).to(device)
# Create a fixed label tensor with a dimension of 1
fixed_labels_1 = torch.full((1,), 2).to(device)  # Change the label value as needed

# Define the number of interpolation steps
num_steps = 10  # You can adjust this number as needed

# Perform linear interpolation between the two latent vectors
# interpolated_latent_vectors = [latent_vector1 + (i / num_steps) * (latent_vector2 - latent_vector1) for i in range(num_steps)]

# Interpolate between the latent representations
interpolated_latents = []
for alpha in torch.linspace(0, 1, num_steps):
    interpolated_latent = alpha * latent_vector1 + (1 - alpha) * latent_vector2
    interpolated_latents.append(interpolated_latent)


# Generate images from the interpolated latent vectors
generated_images = []
for i, latent_vector in enumerate(interpolated_latents):
    with torch.no_grad():
        fake_image = gen(latent_vector, fixed_labels_1)
        generated_images.append(fake_image)

plt.figure(figsize=(12, 6))
for i in range(num_steps):
    plt.subplot(2, num_steps // 2, i+1)
    plt.imshow(generated_images[i][0].cpu().numpy().transpose(1, 2, 0))
    plt.axis('off')
    plt.title(f'Step {i+1}')

plt.show()


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np
from scipy.linalg import sqrtm

# Load a pre-trained Inception model
from torchvision.models import inception_v3

# Load the Inception-V3 model without using deprecated parameters
inception_model = inception_v3(pretrained=True, num_classes=1000)
inception_model.aux_logits = False  # Disable auxiliary logits
inception_model.fc = nn.Identity()  # Remove the final classification layer

def extract_inception_features(images, batch_size=32):
    transform = transforms.Compose([transforms.Resize((299, 299))])
    dataset = torch.utils.data.TensorDataset(images, torch.zeros(len(images)))  # Create a dummy target
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    inception_features = []

    inception_model.eval()
    with torch.no_grad():
        for batch in dataloader:
            batch_images, _ = batch
            batch_images = transform(batch_images)  # Resize the images to 299x299
            batch_features = inception_model(batch_images)
            inception_features.append(batch_features)

    inception_features = torch.cat(inception_features, 0)
    return inception_features.cpu().numpy()


# Function to calculate FID
def calculate_fid(real_features, gen_features):
    mu_real = np.mean(real_features, axis=0)
    mu_gen = np.mean(gen_features, axis=0)
    
    cov_real = np.cov(real_features, rowvar=False)
    cov_gen = np.cov(gen_features, rowvar=False)
    
    cov_sqrt = sqrtm(np.dot(cov_real, cov_gen))
    
    fid = np.sum((mu_real - mu_gen) ** 2) + np.trace(cov_real + cov_gen - 2 * cov_sqrt)
    
    return fid



In [ ]:
fixed_noise = torch.randn(1000, params['nz'], 1, 1).to(device)
fixed_labels_0  = torch.full((1000,), 0).to(device)
fixed_labels_1  = torch.full((1000,), 1).to(device)
fixed_labels_2  = torch.full((1000,), 2).to(device)

# Generate images
with torch.no_grad():
    fake_0 = gen(fixed_noise, fixed_labels_0)
    fake_1 = gen(fixed_noise, fixed_labels_1)
    fake_2 = gen(fixed_noise, fixed_labels_2)



desired_labels = [0, 1, 2]
num_collected_images = {label: 0 for label in desired_labels}
collected_data = {label: {'images': [], 'labels': []} for label in desired_labels}

for batch_images, batch_labels in train_dataloader:
    for label in desired_labels:
        # Find indices where labels are equal to the desired label
        mask = (batch_labels == label).nonzero(as_tuple=True)[0]

        if len(mask) > 0:
            # Select only the samples with the desired label
            selected_images = batch_images[mask]
            selected_labels = batch_labels[mask]

            # Append the batch of images and labels to the collected lists for the current label
            collected_data[label]['images'].append(selected_images)
            collected_data[label]['labels'].append(selected_labels)

            # Update the number of collected images for the current label
            num_collected_images[label] += len(selected_images)

        # # Check if you have collected enough images for the current label (e.g., 1000 for each label)
        # if num_collected_images[label] >= 1000:
        #     break

# Access collected data for label 0
data_for_label_0 = collected_data[0]
real_0 = data_for_label_0['images']
real_label_0 = data_for_label_0['labels']

# Access collected data for label 1
data_for_label_1 = collected_data[1]
real_1 = data_for_label_1['images']
real_label_1 = data_for_label_1['labels']

# Access collected data for label 2
data_for_label_2 = collected_data[2]
real_2 = data_for_label_2['images']
real_label_2 = data_for_label_2['labels']

# Now you have separate collected_data for labels 0, 1, and 2, each containing 1000 or fewer samples.

# Concatenate the collected images and labels into tensors
real_0 = torch.cat(real_0)[:1000]
real_1 = torch.cat(real_1)[:1000]
real_2 = torch.cat(real_2)[:1000]
# real_images = torch.cat(real_labels)[:1000]

# Assuming real_images and gen_images are PyTorch tensors or numpy arrays
# Extract features from real and generated images

inception_model = inception_model.to(device)
real_0 = real_0.to(device)
fake_0 = fake_0.to(device)

real_features = extract_inception_features(real_0)
gen_features = extract_inception_features(fake_0)

# Calculate FID
fid = calculate_fid(real_features, gen_features)
print(f"FID 0: {np.real(fid)}")

inception_model = inception_model.to(device)
real_1 = real_1.to(device)
fake_1 = fake_1.to(device)

real_2 = real_2.to(device)
fake_2 = fake_2.to(device)

real_features = extract_inception_features(real_1)
gen_features = extract_inception_features(fake_1)

# Calculate FID
fid = calculate_fid(real_features, gen_features)
print(f"FID 1 : {np.real(fid)}")


real_features = extract_inception_features(real_2)
gen_features = extract_inception_features(fake_2)

# Calculate FID
fid = calculate_fid(real_features, gen_features)
print(f"FID 2 : {np.real(fid)}")



### Model Training with LABEL as input
- **Loss Curves:**
  - Plotted generator and critic loss curves to monitor convergence and stability.

- **Generated Images:**
  - Displayed 10x10 grids of generated images for each class.

### Evaluation
- **FID Scores:**
  - Calculated class-specific FID scores:
    - Class 0: 140.39
    - Class 1: 134.25
    - Class 2: 171.90
  - Assess the match between real and generated image distributions.

- **Interpolation Results:**
  - Generated images through linear interpolation for visual diversity.

### Observations
- **Training Stability:**
  - Evaluated stability during training.

- **Image Quality:**
  - Assessed the quality of generated images.
  - It was observed that FID of wild image is significantly higher than other two. 
  - Possible reason may be that object detection of wild images are much more complex than other two.

  

### Future Considerations
- **Improvement Areas:**
  - Identified potential areas for model enhancement.


### CWGAN AS LABEL ONE HOT ENCODER

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.transforms as transformations
from torch.utils.data import DataLoader
import torchvision.utils as vutils
import PIL.Image as Image
import torch.nn.functional as F
import torchvision.models as models
from torchvision.datasets import ImageFolder
import numpy as np
from tqdm import tqdm
from ignite.metrics.gan import FID
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Parameters to define the model.
params = {
    "bsize" : 64,# Batch size during training.
    'imsize' : 128,# Spatial size of training images. All images will be resized to this size during preprocessing.
    'nc' : 3,# Number of channles in the training images. For coloured images this is 3.
    'nz' : 100,# Size of the Z latent vector (the input to the generator).
    'ngf' : 64,# Size of feature maps in the generator. The depth will be multiples of this.
    'ndf' : 64, # Size of features maps in the discriminator. The depth will be multiples of this.
    'nepochs' : 200,# Number of training epochs.
    'lr' : 1e-4,# Learning rate for optimizers
    'beta1' : 0.5,# Beta1 hyperparam for Adam optimizer
    'num_classes' : 3,# number of label in animal dataset
    'gen_embs' : 100, # embedding of labels
    'critic_iter' : 5, # number of critic iteration
    'lambda_gp' : 10 # Gradient penality
    } 


In [ ]:
!gpustat

In [ ]:
device = torch.device("cuda:2" if(torch.cuda.is_available()) else "cpu")
print(device, " will be used.\n")

In [ ]:
# Define the path to your dataset and other hyperparameters
data_dir_train = '/data4/home/anujkumar1/GAN/Datasets/images/train' 
data_dir_val = '/data4/home/anujkumar1/GAN/Datasets/images/val'
shuffle_dataset = True  
num_workers = 4  


transform = transforms.Compose([
    transforms.Resize((params['imsize'],params['imsize'])), 
    transforms.ToTensor(),  
])


train_dataset = ImageFolder(root= data_dir_train, transform=transform)
val_dataset = ImageFolder(root= data_dir_val , transform=transform)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=params['bsize'],
    shuffle=shuffle_dataset,
    num_workers=num_workers
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=100,
    shuffle=shuffle_dataset,
    num_workers=num_workers
)


In [ ]:
class Generator(nn.Module):
    def __init__(self, params):
        super(Generator, self).__init__()
        self.img_size = params['imsize']
        self.num_classes = params['num_classes']
        self.embed_size = params['gen_embs']
        nz = params['nz']
        ngf = params['ngf']
        nc = params['nc']
        
        self.main = nn.Sequential(
            # Input: N x channels_noise x 1 x 1
            nn.ConvTranspose2d(nz + self.num_classes, ngf * 32, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 32),
            nn.ReLU(),
             # img: 4x4
            nn.ConvTranspose2d(ngf * 32, ngf * 16, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 16),
            nn.ReLU(),
            # img: 8x8
            nn.ConvTranspose2d(ngf * 16, ngf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(),
            # img: 16x16
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(),
            # img: 32x32
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(),
            # img: 64x64
            nn.ConvTranspose2d(ngf * 2, nc, kernel_size=4, stride=2, padding=1),
            # Output: N x channels_img x 128 x 128
            nn.Tanh(),
        )

        self.embed = nn.Embedding(self.num_classes, self.embed_size)

    def forward(self, x, labels):
        print('labes', labels.shape)
        print('x:',x.shape)
        labels = labels.unsqueeze(-1).unsqueeze(-1)
        # embedding = self.embed(labels).unsqueeze(2).unsqueeze(3)
        # print('embd:', embedding)
        x = torch.cat([x, labels], 1)
        print('x1:', x.shape)
        print (self.main(x).shape)
        return self.main(x)





In [ ]:
class Discriminator(nn.Module):
    def __init__(self, params):
        super(Discriminator, self).__init__()
        self.img_size = params['imsize']
        nc = params['nc']
        ndf = params['ndf']
        num_classes = params['num_classes']

        self.main = nn.Sequential(
            nn.Conv2d(nc + num_classes, ndf, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),

            # Block 1
            nn.Conv2d(ndf, ndf * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(ndf * 2, affine=True),
            nn.LeakyReLU(0.2),

            # Block 2
            nn.Conv2d(ndf * 2, ndf * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(ndf * 4, affine=True),
            nn.LeakyReLU(0.2),

            # Block 3
            nn.Conv2d(ndf * 4, ndf * 8, kernel_size=4, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(ndf * 8, affine=True),
            nn.LeakyReLU(0.2),

             # Block 4
            nn.Conv2d(ndf * 8, ndf * 16, kernel_size=4, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(ndf * 16, affine=True),
            nn.LeakyReLU(0.2),

            # Output
            nn.Conv2d(ndf * 16, 1, kernel_size=4, stride=2, padding=0),
        )

        # Embedding for conditioning
        self.embed = nn.Embedding(num_classes, self.img_size * self.img_size)

    def forward(self, x, labels):
        # print('x:', x.shape)
        # print('lab:', labels.shape)
        embedding = self.embed(labels).view(labels.shape[0], 3, self.img_size, self.img_size)
        # print('embs: ', embedding.shape)
        x = torch.cat([x, embedding], dim=1)
        # print('x1:', x.shape)
        # print('x2', self.main(x).shape)
        return self.main(x)




In [ ]:
# des1 = Encoder(params).to(device)
des3 = Discriminator(params).to(device)
des2 = Generator(params).to(device)
for x,labels in train_dataloader:
    x=x.to(device)
    z = torch.randn(100,100,1,1).to(device)
    labels = F.one_hot(labels, params['num_classes']).to(device)
    fixed_labels_0 = torch.full((100,), 0)
    fixed_labels_0 = F.one_hot(fixed_labels_0, params['num_classes']).to(device)
    # print(des2(z,labels).shape)
    print(des2(z,fixed_labels_0).shape)
    # print(des3(x,labels).shape)
    break

In [ ]:
def weights_init(w):
    """
    Initializes the weights of the layer, w.
    """
    classname = w.__class__.__name__
    if classname.find('conv') != -1:
        nn.init.normal_(w.weight.data, 0.0, 0.02)
    elif classname.find('bn') != -1:
        nn.init.normal_(w.weight.data, 1.0, 0.02)
        nn.init.constant_(w.bias.data, 0)
        

In [ ]:
# Create the generator model with the provided parameters.
gen = Generator(params).to(device)
gen.apply(weights_init)
critic = Discriminator(params).to(device)
critic.apply(weights_init), gen

In [ ]:
# initializate optimizer
opt_gen = optim.Adam(gen.parameters(), lr=params['lr'], betas=(params['beta1'], 0.999))
opt_critic = optim.Adam(critic.parameters(), lr=params['lr'], betas=(params['beta1'], 0.999))

In [ ]:
fixed_noise = torch.randn(100, params['nz'], 1, 1).to(device)
fixed_labels = torch.randint(0, 3, (100,))
fixed_labels = F.one_hot(fixed_labels, params['num_classes']).to(device)


In [ ]:
### initialize FID wrapper
fid_score = FID()
#interpolate function to resize images to 299,299,3  which is the input size of inception network
def interpolate(batch):
    arr = []
    for img in batch:
        pil_img = transformations.ToPILImage()(img)
        resized_img = pil_img.resize((299,299), Image.BILINEAR)
        arr.append(transformations.ToTensor()(resized_img))
    return torch.stack(arr)

In [ ]:
# # Load the Generator model
# gen.load_state_dict(torch.load('/data4/home/anujkumar1/GAN/generator_model.pth'))
# # gen.eval()

# # Load the Discriminator model
# critic.load_state_dict(torch.load('/data4/home/anujkumar1/GAN/critic_model.pth'))
# # critic.eval()


In [ ]:
gen.train()
critic.train()
#### gradient penalty function for WGAN-GP
def gradient_penalty(critic,labels, real, fake, device="cpu"):
    BATCH_SIZE, C, H, W = real.shape
    alpha = torch.rand((BATCH_SIZE, 1, 1, 1)).repeat(1, C, H, W).to(device)
    interpolated_images = real * alpha + fake * (1 - alpha)

    # Calculate critic scores
    mixed_scores = critic(interpolated_images, labels)

    # Take the gradient of the scores with respect to the images
    gradient = torch.autograd.grad(
        inputs=interpolated_images,
        outputs=mixed_scores,
        grad_outputs=torch.ones_like(mixed_scores),
        create_graph=True,
        retain_graph=True,
    )[0]
    gradient = gradient.view(gradient.shape[0], -1)
    gradient_norm = gradient.norm(2, dim=1)
    gradient_penalty = torch.mean((gradient_norm - 1) ** 2)
    return gradient_penalty



In [ ]:
step = 0

G_loss = []
C_loss = []

for epoch in range(params['nepochs']):
    #we will track the total loss of the generator and critic for each epoch over the entire dataset
    #initialize the total loss of the generator and critic for each epoch to 0
    total_loss_gen = 0
    total_loss_critic = 0
    
    #move these to device

    #have no gradient for these losse
    # total_loss_gen = torch.no_grad()
    # total_loss_critic = torch.no_grad()
    
    # Target labels 
    for batch_idx, (real, labels) in enumerate(tqdm(train_dataloader)):
        #send labels to device
        labels = F.one_hot(labels, params['num_classes'])
        labels = labels.to(device)
        batch_step = 0
        real = real.to(device)
        cur_batch_size = real.shape[0]
        
        

        # Train Critic: 
        for _ in range(params['critic_iter']):
            noise = torch.randn(cur_batch_size, params['nz'], 1, 1).to(device)
            if len(noise) != len(labels):
                noise = noise[:len(labels)]
            fake = gen(noise, labels)
            critic_real = critic(real, labels).reshape(-1)
            critic_fake = critic(fake, labels).reshape(-1)
            gp = gradient_penalty(critic, labels, real, fake, device=device)
            loss_critic = (
                -(torch.mean(critic_real) - torch.mean(critic_fake)) + params['lambda_gp'] * gp
            )
            
            
            critic.zero_grad()
            loss_critic.backward(retain_graph=True)
            opt_critic.step()
            
        #trained critic
        

        # Train Generator: 
        gen_fake = critic(fake, labels).reshape(-1)
        loss_gen = -torch.mean(gen_fake)
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()
        
        
        
        
        
        #update the total loss of the generator and critic for each batch in the epoch
        #add just the value no gradients
        #have no gradient for these losse
        #add just the value no gradientsfrom the loss_gen tensor
      
        with torch.no_grad():
            total_loss_gen += loss_gen.item()
            total_loss_critic += loss_critic.item()
        
        
        # # Print losses occasionally and print to tensorboard in a batch
        # if (batch_idx + 1) % 23 == 0 and batch_idx > 0:
        #     print(
        #         f"Epoch [{epoch+1}/{params['nepochs']}] Batch {batch_idx+1}/{len(train_dataloader)} \
        #           Loss D: {loss_critic:.4f}, loss G: {loss_gen:.4f}"
        #     )
            
        batch_step += 1
    
        G_loss.append(loss_gen.item())
        C_loss.append(loss_critic.item())   

    print(f"Epoch [{epoch+1}/{params['nepochs']}] Batch {batch_idx+1}/{len(train_dataloader)} \
                Loss D: {loss_critic:.4f}, loss G: {loss_gen:.4f}"
            )

    if (epoch+1)% 20 == 0:
        
        with torch.no_grad():
            fake = gen(fixed_noise, fixed_labels)
            # take out (up to) 32 examples
            grid = vutils.make_grid(fake, nrow= 10, padding=2, normalize=True)[:100].cpu().numpy()
            # Display the generated image grid with a larger size
            plt.figure(figsize=(12, 12))  # Adjust the figsize as needed
            plt.axis("off")
            plt.title("Generated Images")
            plt.imshow(np.transpose(grid[:100], (1, 2, 0)))
            plt.show()


        #AVERAGE LOSS---

        #get average loss of generator and critic for each epoch
        avg_loss_gen = total_loss_gen / len(train_dataloader)
        avg_loss_critic = total_loss_critic / len(train_dataloader)

    
    step += 1
        

In [ ]:
# Plot the training losses.
plt.figure(figsize=(10,5))
plt.title("Generator and critic Loss During Training")
plt.plot(G_loss,label="G")
plt.plot(C_loss,label="C")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()



In [ ]:
# # Save the Generator model
# torch.save(gen.state_dict(), 'generator_model.pth')

# # Save the Discriminator model
# torch.save(critic.state_dict(), 'critic_model.pth')


In [ ]:
# Save the Generator model
torch.save(gen.state_dict(), 'generator_model_one_hot.pth')

# Save the Discriminator model
torch.save(critic.state_dict(), 'critic_model_one_hot.pth')


In [ ]:
# Load the Generator model
# gen.load_state_dict(torch.load('/data4/home/anujkumar1/GAN/generator_model.pth'))
gen.eval()

# Load the Discriminator model
# critic.load_state_dict(torch.load('/data4/home/anujkumar1/GAN/critic_model.pth'))
critic.eval()


In [ ]:
fixed_noise = torch.randn(100, params['nz'], 1, 1).to(device)
fixed_labels_0 = torch.full((100,), 0)
fixed_labels_0 = F.one_hot(fixed_labels_0, params['num_classes']).to(device)
print(gen(fixed_noise,fixed_labels_0).shape)
    

In [ ]:
import torch

fixed_labels_2 = torch.full((100, 3), 2)
print(fixed_labels_2.size())


In [ ]:
fixed_noise = torch.randn(100, params['nz'], 1, 1).to(device)
fixed_labels_0 = torch.full((100,), 0)
print(fixed_labels_0.shape)
fixed_labels_0 = F.one_hot(fixed_labels_0, params['num_classes']).to(device)
print(fixed_labels_0.shape)
fixed_labels_1 = torch.full((100,), 1)
fixed_labels_1 = F.one_hot(fixed_labels_1, params['num_classes']).to(device)
fixed_labels_2 = torch.full((100,), 2)
fixed_labels_2 = F.one_hot(fixed_labels_2, params['num_classes']).to(device)

# Generate images
with torch.no_grad():
    print(fixed_noise.shape)
    print(fixed_labels_0.shape)
    fake_0 = gen(fixed_noise, fixed_labels_0)
    fake_1 = gen(fixed_noise, fixed_labels_1)
    fake_2 = gen(fixed_noise, fixed_labels_2)



In [ ]:
# Reshape the generated images into a 10x10 grid
grid_0 = vutils.make_grid(fake_0, nrow= 10, padding=2, normalize=True)[:100].cpu().numpy()
grid_1 = vutils.make_grid(fake_1, nrow= 10, padding=2, normalize=True)[:100].cpu().numpy()
grid_2 = vutils.make_grid(fake_2, nrow= 10, padding=2, normalize=True)[:100].cpu().numpy()

# Display the generated image grid with a larger size
plt.figure(figsize=(12, 12))  # Adjust the figsize as needed
plt.axis("off")
plt.title("Generated Images_0")
plt.imshow(np.transpose(grid_0[:100], (1, 2, 0)))
plt.show()

# Display the generated image grid with a larger size
plt.figure(figsize=(12, 12))  # Adjust the figsize as needed
plt.axis("off")
plt.title("Generated Images_1")
plt.imshow(np.transpose(grid_1, (1, 2, 0)))
plt.show()

# Display the generated image grid with a larger size
plt.figure(figsize=(12, 12))  # Adjust the figsize as needed
plt.axis("off")
plt.title("Generated Images_2")
plt.imshow(np.transpose(grid_2, (1, 2, 0)))
plt.show()

In [ ]:
fixed_noise = torch.randn(1000, params['nz'], 1, 1).to(device)
fixed_labels_0 = torch.full((1000,), 0)
fixed_labels_0 = F.one_hot(fixed_labels_0, params['num_classes']).to(device)
fixed_labels_1 = torch.full((1000,), 1)
fixed_labels_1 = F.one_hot(fixed_labels_1, params['num_classes']).to(device)
fixed_labels_2 = torch.full((1000,), 2)
fixed_labels_2 = F.one_hot(fixed_labels_2, params['num_classes']).to(device)


# Generate images
with torch.no_grad():
    fake_0 = gen(fixed_noise, fixed_labels_0)
    fake_1 = gen(fixed_noise, fixed_labels_1)
    fake_2 = gen(fixed_noise, fixed_labels_2)



real = []
real_labels = []

for batch_images, batch_labels in val_dataloader:
    # Append the batch of images and labels to the collected lists
    real.append(batch_images)
    real_labels.append(batch_labels)

    # Check if you have collected enough images
    num_collected_images = sum(len(images) for images in real)
    if num_collected_images >= 1000:
        break

# Concatenate the collected images and labels into tensors
real = torch.cat(real)[:1000]
real_images = torch.cat(real_labels)[:1000]

In [ ]:
# Define the two latent vectors for interpolation
latent_vector1 = torch.randn(1, params['nz'], 1, 1).to(device)
latent_vector2 = torch.randn(1, params['nz'], 1, 1).to(device)
# Create a fixed label tensor with a dimension of 1
fixed_labels_1 = torch.full((1,), 1).to(device)  # Change the label value as needed
fixed_labels_1 = F.one_hot(fixed_labels_1, params['num_classes']).to(device)
# Define the number of interpolation steps
num_steps = 10  # You can adjust this number as needed

# Perform linear interpolation between the two latent vectors
# interpolated_latent_vectors = [latent_vector1 + (i / num_steps) * (latent_vector2 - latent_vector1) for i in range(num_steps)]

# Interpolate between the latent representations
interpolated_latents = []
for alpha in torch.linspace(0, 1, num_steps):
    interpolated_latent = alpha * latent_vector1 + (1 - alpha) * latent_vector2
    interpolated_latents.append(interpolated_latent)


# Generate images from the interpolated latent vectors
generated_images = []
for i, latent_vector in enumerate(interpolated_latents):
    with torch.no_grad():
        fake_image = gen(latent_vector, fixed_labels_1)
        generated_images.append(fake_image)

plt.figure(figsize=(12, 6))
for i in range(num_steps):
    plt.subplot(2, num_steps // 2, i+1)
    plt.imshow(generated_images[i][0].cpu().numpy().transpose(1, 2, 0))
    plt.axis('off')
    plt.title(f'Step {i+1}')

plt.show()


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np
from scipy.linalg import sqrtm

# Load a pre-trained Inception model
from torchvision.models import inception_v3

# Load the Inception-V3 model without using deprecated parameters
inception_model = inception_v3(pretrained=True, num_classes=1000)
inception_model.aux_logits = False  # Disable auxiliary logits
inception_model.fc = nn.Identity()  # Remove the final classification layer

def extract_inception_features(images, batch_size=32):
    transform = transforms.Compose([transforms.Resize((299, 299))])
    dataset = torch.utils.data.TensorDataset(images, torch.zeros(len(images)))  # Create a dummy target
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    inception_features = []

    inception_model.eval()
    with torch.no_grad():
        for batch in dataloader:
            batch_images, _ = batch
            batch_images = transform(batch_images)  # Resize the images to 299x299
            batch_features = inception_model(batch_images)
            inception_features.append(batch_features)

    inception_features = torch.cat(inception_features, 0)
    return inception_features.cpu().numpy()


# Function to calculate FID
def calculate_fid(real_features, gen_features):
    mu_real = np.mean(real_features, axis=0)
    mu_gen = np.mean(gen_features, axis=0)
    
    cov_real = np.cov(real_features, rowvar=False)
    cov_gen = np.cov(gen_features, rowvar=False)
    
    cov_sqrt = sqrtm(np.dot(cov_real, cov_gen))
    
    fid = np.sum((mu_real - mu_gen) ** 2) + np.trace(cov_real + cov_gen - 2 * cov_sqrt)
    
    return fid



In [ ]:
fixed_noise = torch.randn(1000, params['nz'], 1, 1).to(device)
fixed_labels_0 = torch.full((1000,), 0)
fixed_labels_0 = F.one_hot(fixed_labels_0, params['num_classes']).to(device)
fixed_labels_1 = torch.full((1000,), 1)
fixed_labels_1 = F.one_hot(fixed_labels_1, params['num_classes']).to(device)
fixed_labels_2 = torch.full((1000,), 2)
fixed_labels_2 = F.one_hot(fixed_labels_2, params['num_classes']).to(device)

with torch.no_grad():
    fake_0 = gen(fixed_noise, fixed_labels_0)
    fake_1 = gen(fixed_noise, fixed_labels_1)
    fake_2 = gen(fixed_noise, fixed_labels_2)



desired_labels = [0, 1, 2]
num_collected_images = {label: 0 for label in desired_labels}
collected_data = {label: {'images': [], 'labels': []} for label in desired_labels}

for batch_images, batch_labels in train_dataloader:
    for label in desired_labels:
        # Find indices where labels are equal to the desired label
        mask = (batch_labels == label).nonzero(as_tuple=True)[0]

        if len(mask) > 0:
            # Select only the samples with the desired label
            selected_images = batch_images[mask]
            selected_labels = batch_labels[mask]

            # Append the batch of images and labels to the collected lists for the current label
            collected_data[label]['images'].append(selected_images)
            collected_data[label]['labels'].append(selected_labels)

            # Update the number of collected images for the current label
            num_collected_images[label] += len(selected_images)

        # # Check if you have collected enough images for the current label (e.g., 1000 for each label)
        # if num_collected_images[label] >= 1000:
        #     break

# Access collected data for label 0
data_for_label_0 = collected_data[0]
real_0 = data_for_label_0['images']
real_label_0 = data_for_label_0['labels']

# Access collected data for label 1
data_for_label_1 = collected_data[1]
real_1 = data_for_label_1['images']
real_label_1 = data_for_label_1['labels']

# Access collected data for label 2
data_for_label_2 = collected_data[2]
real_2 = data_for_label_2['images']
real_label_2 = data_for_label_2['labels']

# Now you have separate collected_data for labels 0, 1, and 2, each containing 1000 or fewer samples.

# Concatenate the collected images and labels into tensors
real_0 = torch.cat(real_0)[:1000]
real_1 = torch.cat(real_1)[:1000]
real_2 = torch.cat(real_2)[:1000]
# real_images = torch.cat(real_labels)[:1000]

# Assuming real_images and gen_images are PyTorch tensors or numpy arrays
# Extract features from real and generated images

inception_model = inception_model.to(device)
real_0 = real_0.to(device)
fake_0 = fake_0.to(device)

real_features = extract_inception_features(real_0)
gen_features = extract_inception_features(fake_0)

# Calculate FID
fid = calculate_fid(real_features, gen_features)
print(f"FID 0: {np.real(fid)}")

inception_model = inception_model.to(device)
real_1 = real_1.to(device)
fake_1 = fake_1.to(device)

real_2 = real_2.to(device)
fake_2 = fake_2.to(device)

real_features = extract_inception_features(real_1)
gen_features = extract_inception_features(fake_1)

# Calculate FID
fid = calculate_fid(real_features, gen_features)
print(f"FID 1 : {np.real(fid)}")


real_features = extract_inception_features(real_2)
gen_features = extract_inception_features(fake_2)

# Calculate FID
fid = calculate_fid(real_features, gen_features)
print(f"FID 2 : {np.real(fid)}")

### Model Training with LABEL ONE-HOT ENCODER as input
- **Loss Curves:**
  - Plotted generator and critic loss curves to track convergence.

- **Generated Images:**
  - Showcased 10x10 grids of generated images for each class.

### Evaluation
- **FID Scores:**
  - Calculated class-specific FID scores:
    - Class 0: 148.39
    - Class 1: 144.09
    - Class 2: 175.76
  - Assessed agreement between real and generated distributions.

- **Interpolation Results:**
  - Generated images through linear interpolation for visual exploration.

### Observations
- **Training Stability:**
  - Examined stability and robustness during training.

- **Image Quality:**
  - Evaluated the quality of generated images.
  - It was observed that FID of wild image is significantly higher than other two. 
  - Possible reason may be that object detection of wild images are much more complex than other two.

- **Obseravtions between two label inputs**
  - There is no significant change in FID score whether label input is one hot encoder or label as embedding
  

### Future Considerations
- **Enhancements:**
  - Identified potential areas for model improvement.


### BIGAN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.transforms as transformations
from torch.utils.data import DataLoader
import torchvision.utils as vutils
import torch.nn.functional as F
from torch.nn import init, Module, ReLU, Sequential, ModuleList, Conv2d, MaxPool2d, LeakyReLU, Flatten, Linear, Sigmoid, ConvTranspose2d, BatchNorm2d, Tanh, Dropout2d, Dropout
from torch.nn.functional import binary_cross_entropy as bce
import PIL.Image as Image
import torchvision.models as models
from torchvision.datasets import ImageFolder
import numpy as np
from tqdm import tqdm
from ignite.metrics.gan import FID
import matplotlib.pyplot as plt

In [ ]:
# Parameters to define the model.
params = {
    "bsize" : 128,# Batch size during training.
    'imsize' : 128,# Spatial size of training images. All images will be resized to this size during preprocessing.
    'nc' : 3,# Number of channles in the training images. For coloured images this is 3.
    'nz' : 100,# Size of the Z latent vector (the input to the generator).
    'ngf' : 64,# Size of feature maps in the generator. The depth will be multiples of this.
    'ndf' : 64, # Size of features maps in the discriminator. The depth will be multiples of this.
    'nepochs' : 200,# Number of training epochs.
    'lr' : 1e-4,# Learning rate for optimizers
    'beta1' : 0.5,# Beta1 hyperparam for Adam optimizer
    'num_classes' : 3,# number of label in animal dataset
    'gen_embs' : 100, # embedding of labels
    'critic_iter' : 5, # number of critic iteration
    'lambda_gp' : 10 # Gradient penality
    } 


In [ ]:
device = torch.device("cuda:4" if(torch.cuda.is_available()) else "cpu")
print(device, " will be used.\n")

In [ ]:
# Define the path to your dataset and other hyperparameters
data_dir_train = '/data4/home/anujkumar1/GAN/Datasets/images/train' 
data_dir_val = '/data4/home/anujkumar1/GAN/Datasets/images/val'
shuffle_dataset = True  
num_workers = 4  


transform = transforms.Compose([
    transforms.Resize((params['imsize'],params['imsize'])), 
    transforms.ToTensor(),  
])


train_dataset = ImageFolder(root= data_dir_train, transform=transform)
val_dataset = ImageFolder(root= data_dir_val , transform=transform)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=params['bsize'],
    shuffle=shuffle_dataset,
    num_workers=num_workers
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=100,
    shuffle=shuffle_dataset,
    num_workers=num_workers
)


In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
        if m.bias is not None:
            m.bias.data.fill_(0)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
    elif classname.find('Linear') != -1:
        m.bias.data.fill_(0)

In [ ]:
class Encoder(nn.Module):
    def __init__(self,params):
        super(Encoder, self).__init__()
        ndf = params['ndf']
        nc = params['nc']

        self.conv_blocks = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf),
            nn.ReLU(True),

            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.ReLU(True),

            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.ReLU(True),

            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.ReLU(True),

            nn.Conv2d(ndf * 8, ndf * 16, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 16),
            nn.ReLU(True),

            nn.Conv2d(ndf * 16, ndf * 16, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ndf * 16),
            nn.ReLU(True),

            nn.Conv2d(ndf * 16, 100, 1, 1)
        )

    def forward(self, x):
        x = self.conv_blocks(x)
        return x


In [ ]:
class Generator(nn.Module):
    def __init__(self,params):
        super(Generator, self).__init__()
        ngf = params['ngf']
        nz = params['nz']

        self.conv_blocks = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 16, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 16),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(ngf * 16, ngf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(ngf, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.conv_blocks(x)
        return x


In [ ]:
class Discriminator(nn.Module):
    def __init__(self,params):
        super(Discriminator, self).__init__()
        ndf = params['ndf']
        nc = params['nc']
        nz = params['nz']

        self.conv_blocks_x = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1),
            nn.LeakyReLU(0.2),

            nn.Conv2d(ndf, ndf * 2, 4, 2, 1),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2),

            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2),

            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2),

            nn.Conv2d(ndf * 8, ndf * 8, 4, 2, 1),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2),

            nn.Conv2d(ndf * 8, ndf * 8, 4, 1, 0),
            nn.LeakyReLU(0.2)
        )

        self.conv_blocks_z = nn.Sequential(
            nn.Conv2d(nz, ndf * 8, 1, 1, 0),
            nn.LeakyReLU(0.2),

            nn.Conv2d(ndf * 8, ndf * 8, 1, 1, 0),
            nn.LeakyReLU(0.2)
        )

        self.conv_blocks_combined = nn.Sequential(
            nn.Conv2d(ndf * 16, ndf * 32, 1, 1, 0),
            nn.LeakyReLU(0.2),

            nn.Conv2d(ndf * 32, ndf * 32, 1, 1, 0),
            nn.LeakyReLU(0.2),

            nn.Conv2d(ndf * 32, 1, 1, 1, 0),
            nn.Sigmoid()
        )

    def forward(self, x, z):
        x = self.conv_blocks_x(x)
        z = self.conv_blocks_z(z)
        xz = torch.cat((x, z), dim=1)
        xz = self.conv_blocks_combined(xz)
        return xz


In [ ]:
des1 = Encoder(params).to(device)
des3 = Discriminator(params).to(device)
des2 = Generator(params).to(device)
for x,_ in train_dataloader:
    x=x.to(device)
    z = torch.randn(params['bsize'],100,1,1).to(device)
    print(des1(x).shape)
    print(des2(z).shape)
    print(des3(x,z).shape)
    break

In [ ]:
num_epochs = params['nepochs']

latent_dim = params['nz']

# discriminator = Discriminator().to(device)
# generator =  Generator().to(device)
# encoder = Encoder().to(device)

# discriminator.apply(weights_init)
# encoder.apply(weights_init)
# generator.apply(weights_init)


model = nn.Sequential(Discriminator(params), Generator(params), Encoder(params)).to(device)
model.apply(weights_init)



optim_generator = torch.optim.Adam([{'params' : model[1].parameters()},
                        {'params' : model[2].parameters()}],lr=0.0002,betas=(0.5,0.999))
optim_discriminator = torch.optim.Adam(model[0].parameters(),lr=0.0002,betas=(0.5,0.999))


In [ ]:
def sample(k,Z=params['nz']):
    return torch.randn((k, Z, 1, 1)).to(device)

In [ ]:
for epoch in range(num_epochs):
    i = 0
    d_loss = 0.0
    ge_loss  = 0.0

    # num_training_steps = len(train_dataloader)
    # progress_bar = tqdm(range(num_training_steps))

    for batch_idx, (X,_) in enumerate(tqdm(train_dataloader)):


        x_real = X.to(device)
        z_real = sample(params['bsize'])

        z_fake, x_fake = model[2](x_real), model[1](z_real)
        real_preds, fake_preds = model[0](x_real, z_fake), model[0](x_fake, z_real)

        disc_loss = F.binary_cross_entropy(real_preds.squeeze(), torch.ones((len(real_preds),)).to(device)) + F.binary_cross_entropy(fake_preds.squeeze(), torch.zeros((len(fake_preds),)).to(device))
        gen_loss = F.binary_cross_entropy(real_preds.squeeze(), torch.zeros((len(real_preds),)).to(device)) + F.binary_cross_entropy(fake_preds.squeeze(), torch.ones((len(fake_preds),)).to(device))

        d_loss += disc_loss.item()
        ge_loss += gen_loss.item()

        
        if not i%5 == 0: # train discriminator 5 times & generator once
            optim_discriminator.zero_grad()
            disc_loss.backward()
            optim_discriminator.step()
        else:
            optim_generator.zero_grad()
            gen_loss.backward()
            optim_generator.step()

        i += 1
        
        # progress_bar.update(1)
    

    print("[Epoch %d/%d] [D loss: %f] [GE loss: %f] "
            % (epoch+1, num_epochs, d_loss, ge_loss))

    """
    if (epoch+1)%10 == 0:
        torch.save(G.state_dict() , f"./models1/{epoch+1}_G.pth")
        torch.save(E.state_dict() , f"./models1/{epoch+1}_D.pth")
        torch.save(D.state_dict() , f"./models1/{epoch+1}_E.pth")
    """

    if (epoch+1)%10 == 0:
        z = torch.randn(100, 100, 1, 1).to(device)
        z = model[1](z).cpu()
        # save_image(z,f"/data4/home/anujkumar1/Image/{epoch + 1}_Generated.png", nrow=10)

        
        plt.figure(figsize=(20,20))
        plt.subplot(1,2,1)
        plt.axis("off")
        plt.title("Generated Images")
        plt.imshow(np.transpose(vutils.make_grid(z[:100], padding=5, nrow=10,normalize=True).cpu(),(1,2,0)))
        # plt.savefig(f"./generated1/{epoch+1}_Generated.png")
        plt.show()
        

In [ ]:
# Save the model
torch.save(model.state_dict(), 'Bigan_model_copy1.pth')


In [ ]:
# Load the Generator model
device = torch.device("cuda:3" if(torch.cuda.is_available()) else "cpu")
print(device, " will be used.\n")
model = nn.Sequential(Discriminator(params), Generator(params), Encoder(params)).to('cpu')
model.apply(weights_init)
optim_generator = torch.optim.Adam([{'params' : model[1].parameters()},
                        {'params' : model[2].parameters()}],lr=0.0002,betas=(0.5,0.999))
optim_discriminator = torch.optim.Adam(model[0].parameters(),lr=0.0002,betas=(0.5,0.999))

model.load_state_dict(torch.load('Bigan_model_copy1.pth'))
model.eval()


In [ ]:
model[2]

In [ ]:
z = torch.randn(100, 100, 1, 1).cpu()
with torch.no_grad():
    generated_imgs = model[1](z).detach().cpu()

# Reshape the generated images into a 10x10 grid
grid = vutils.make_grid(generated_imgs, nrow= 10, padding=2, normalize=True)

# Display the generated image grid with a larger size
plt.figure(figsize=(12, 12))  # Adjust the figsize as needed
plt.axis("off")
plt.title("Generated Images")
plt.imshow(np.transpose(grid, (1, 2, 0)))
plt.show()

In [ ]:
X_train = []
y_train= []

for batch_images, batch_labels in train_dataloader:
    # Append the batch of images and labels to the collected lists
    batch_images = batch_images#.to(device)
    z = model[2](batch_images)
    z=z.cpu()
    X_train.append(z)
    y_train.append(batch_labels)

X_val = []
y_val = []

for batch_images, batch_labels in val_dataloader:
    # Append the batch of images and labels to the collected lists
    batch_images = batch_images#.to(device)
    z = model[2](batch_images)
    z=z.cpu()
    X_val.append(z)
    y_val.append(batch_labels)


In [ ]:
X_train = torch.cat(X_train).cpu().detach().numpy()
X_val = torch.cat(X_val).cpu().detach().numpy()
y_train = torch.cat(y_train).cpu().detach().numpy()
y_val = torch.cat(y_val).cpu().detach().numpy()


In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

In [ ]:
# Import the necessary libraries
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Reshape the X_train data to be 2D
X_train = X_train.reshape(X_train.shape[0], -1)
X_val = X_val.reshape(X_val.shape[0], -1)
print(X_train.shape,X_val.shape)
# Create an SVM classifier
clf = SVC(kernel='rbf')

# Train the classifier
clf.fit(X_train, y_train)


# Make predictions on the test data
y_pred = clf.predict(X_val)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy*100)


In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(multi_class='multinomial',solver='lbfgs')
# Train the classifier
clf.fit(X_train, y_train)


# Make predictions on the test data
y_pred = clf.predict(X_val)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy*100)


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np
from scipy.linalg import sqrtm

# Load a pre-trained Inception model
from torchvision.models import inception_v3

# Load the Inception-V3 model without using deprecated parameters
inception_model = inception_v3(pretrained=True, num_classes=1000)
inception_model.aux_logits = False  # Disable auxiliary logits
inception_model.fc = nn.Identity()  # Remove the final classification layer

def extract_inception_features(images, batch_size=32):
    transform = transforms.Compose([transforms.Resize((299, 299))])
    dataset = torch.utils.data.TensorDataset(images, torch.zeros(len(images)))  # Create a dummy target
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    inception_features = []

    inception_model.eval()
    with torch.no_grad():
        for batch in dataloader:
            batch_images, _ = batch
            batch_images = transform(batch_images)  # Resize the images to 299x299
            batch_features = inception_model(batch_images)
            inception_features.append(batch_features)

    inception_features = torch.cat(inception_features, 0)
    return inception_features.numpy()


# Function to calculate FID
def calculate_fid(real_features, gen_features):
    mu_real = np.mean(real_features, axis=0)
    mu_gen = np.mean(gen_features, axis=0)
    
    cov_real = np.cov(real_features, rowvar=False)
    cov_gen = np.cov(gen_features, rowvar=False)
    
    cov_sqrt = sqrtm(np.dot(cov_real, cov_gen))
    
    fid = np.sum((mu_real - mu_gen) ** 2) + np.trace(cov_real + cov_gen - 2 * cov_sqrt)
    
    return fid



In [ ]:
model

In [ ]:
noise = torch.randn(1000, params['nz'], 1, 1, device='cpu')
# Generate images
with torch.no_grad():
    generated_imgs = model[1](noise).detach().cpu()
    
real = []
real_labels = []

for batch_images, batch_labels in val_dataloader:
    # Append the batch of images and labels to the collected lists
    real.append(batch_images)
    real_labels.append(batch_labels)

    # Check if you have collected enough images
    num_collected_images = sum(len(images) for images in real)
    if num_collected_images >= 1000:
        break


real_imgs = torch.cat(real)[:1000]
real_labels = torch.cat(real_labels)[:1000]
real_imgs = real_imgs.detach().cpu()


In [ ]:
# Assuming real_images and gen_images are PyTorch tensors or numpy arrays
# Extract features from real and generated images
real_features = extract_inception_features(real_imgs)
gen_features = extract_inception_features(generated_imgs)

# Calculate FID
fid = calculate_fid(real_features, gen_features)


In [ ]:
print(f"FID: {np.real(fid)}")

1. **Bi-GAN Implementation:**
   - Developed a Bi-GAN model for image generation using the provided dataset, incorporating encoder, generator, and discriminator components.

2. **Image Generation and Visualization:**
   - Trained the model to generate images and plotted a 10x10 grid to showcase the diversity and quality of the generated samples.

3. **Posterior Inference and Classifier Training:**
   - Leveraged the trained generator for posterior inference on real data, extracting latent vectors via the encoder.
   - Trained linear and SVM classifiers using the inferred latent vectors to classify among the three classes.

4. **Classifier Evaluation and FID Score:**
   - Assessed the performance of classifiers on a validation set, considering accuracy, precision, recall, and F1-score.
   - Obtained Accuracy of 90.86% using Loistic Regression
   - Obtained Accuracy of 92.86% using SVM - RBF kernel
   - Utilized the FID score (99.24) to quantify the similarity between generated and real data distributions.

5. **Conclusion and Future Work:**
   - Concluded with insights into the Bi-GAN's image generation capabilities and the effectiveness of inferred latent vectors for downstream classification.
   - Outlined potential directions for future improvements or research based on the obtained results.
